In [1]:
import pyconll #pip3 install this if you don't have it
import torchtext.data as tt
import torch 
import torch.nn as nn
import torch.optim as optim
import time
from tools import EarlyStopping

In [2]:
AFRIKAANS_TRAIN = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-train.conllu'
AFRIKAANS_DEV = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-dev.conllu'
AFRIKAANS_TEST = 'UD_Afrikaans-AfriBooms/af_afribooms-ud-test.conllu'

In [3]:
# from https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb
def make_sentences(path):
    data = pyconll.load_from_file(path)
    sentences = []
    tagged_sentences = []
    for each in data:
        tagged_sentence=[]
        sentence = []
        for token in each:
            if token.upos and token.form:
                tagged_sentence.append(token.upos)
                sentence.append(token.form.lower())
        tagged_sentences.append(tagged_sentence)
        sentences.append(sentence)
    return sentences, tagged_sentences

In [4]:
train_afr_raw, tagged_train_afr_raw = make_sentences(AFRIKAANS_TRAIN)
dev_afr_raw, tagged_dev_afr_raw = make_sentences(AFRIKAANS_DEV)
test_afr_raw, tagged_test_afr_raw = make_sentences(AFRIKAANS_TEST)

In [5]:
print("Tagged sentences in train set: ", len(tagged_train_afr_raw))
print("Tagged words in train set:", len([item for sublist in tagged_train_afr_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in dev set: ", len(tagged_dev_afr_raw))
print("Tagged words in dev set:", len([item for sublist in tagged_dev_afr_raw for item in sublist]))
print(40*'=')
print("Tagged sentences in test set: ", len(tagged_test_afr_raw))
print("Tagged words in test set:", len([item for sublist in tagged_test_afr_raw for item in sublist]))
print(40*'*')
print("Total sentences in dataset:", len(tagged_train_afr_raw)+len(tagged_dev_afr_raw)+len(tagged_dev_afr_raw))

Tagged sentences in train set:  1315
Tagged words in train set: 33894
Tagged sentences in dev set:  194
Tagged words in dev set: 5317
Tagged sentences in test set:  425
Tagged words in test set: 10065
****************************************
Total sentences in dataset: 1703


In [6]:
# from https://github.com/tringm/POSTagger_Pytorch/blob/master/src/util/nlp.py
def build_tag_field(sentences_tokens):
    token_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('tokens', token_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_tokens]
    torch_dataset = tt.Dataset(examples, fields)
    return token_field
    
def build_text_field(sentences_words):
    text_field = tt.Field(tokenize=list, init_token="<bos>", eos_token="<eos>")
    fields = [('text', text_field)]
    examples = [tt.Example.fromlist([t], fields) for t in sentences_words]
    torch_dataset = tt.Dataset(examples, fields)
    return text_field

In [7]:
#fields
train_afr = build_text_field(train_afr_raw)
dev_afr = build_text_field(dev_afr_raw)
test_afr = build_text_field(test_afr_raw)
tagged_train_afr = build_tag_field(tagged_train_afr_raw)
tagged_dev_afr = build_tag_field(tagged_dev_afr_raw)
tagged_test_afr = build_tag_field(tagged_test_afr_raw)

fields_train = (("text", train_afr), ("udtags", tagged_train_afr))
examples_train = [tt.Example.fromlist(item, fields_train) for item in zip(train_afr_raw, tagged_train_afr_raw)]
fields_dev = (("text", dev_afr), ("udtags", tagged_dev_afr))
examples_dev = [tt.Example.fromlist(item, fields_dev) for item in zip(dev_afr_raw, tagged_dev_afr_raw)]
fields_test = (("text", test_afr), ("udtags", tagged_test_afr))
examples_test = [tt.Example.fromlist(item, fields_test) for item in zip(test_afr_raw, tagged_test_afr_raw)]

train_data = tt.Dataset(examples_train, fields_train)
valid_data = tt.Dataset(examples_dev, fields_dev)
test_data = tt.Dataset(examples_test, fields_test)

#build vocabs so that they are shared between splits
train_afr.build_vocab(train_data, valid_data, test_data)
dev_afr.vocab = train_afr.vocab
test_afr.vocab = train_afr.vocab
tagged_train_afr.build_vocab(train_data, valid_data, test_data)
tagged_dev_afr.vocab = tagged_train_afr.vocab
tagged_test_afr.vocab = tagged_train_afr.vocab

In [8]:
print(train_afr)

In [9]:
print(train_data)

In [10]:
len(train_data)

1315

In [11]:
# from https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
#model
batch_size=128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#needs to be tuple of dataset objects
train_iterator, valid_iterator, test_iterator = tt.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = batch_size,
    device = device, sort=False)

In [12]:
# try without dropout first
class BiLSTMTagger(nn.Module):
    #https://github.com/bentrevett/pytorch-pos-tagging/blob/master/1%20-%20BiLSTM%20for%20PoS%20Tagging.ipynb
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx=pad_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional)
        #fully connected layer
        self.fc = nn.Linear((hidden_dim * 2 if bidirectional else hidden_dim), output_dim)
     
    
    def forward(self, text):
        embedded = self.embedding(text)
        outputs, (hidden, cell) = self.lstm(embedded)
        predictions = self.fc(outputs)
        return predictions

In [13]:
in_dim = len(train_afr.vocab)
emb_dim = 100
hid_dim = 128
out_dim = len(tagged_train_afr.vocab)
n_layers = 1
bidirectional = True
pad_index = train_afr.vocab.stoi[train_afr.pad_token]
tag_pad_idx = tagged_train_afr.vocab.stoi[tagged_train_afr.pad_token]

In [14]:
model = BiLSTMTagger(in_dim, emb_dim, hid_dim, out_dim, n_layers, bidirectional, pad_index)
criterion = nn.CrossEntropyLoss(ignore_index = tag_pad_idx)
optimizer = optim.Adam(model.parameters())

In [15]:
def categorical_accuracy(preds, y, tag_pad_idx):
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [16]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        text = batch.text
        tags = batch.udtags
        
        optimizer.zero_grad()       
        predictions = model(text)        
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)
        
        loss = criterion(predictions, tags) 
        acc = categorical_accuracy(predictions, tags, tag_pad_idx)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            tags = batch.udtags
            
            predictions = model(text)
            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)
            
            loss = criterion(predictions, tags)
            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [19]:
N_EPOCHS = 50
patience = 3
early_stopping = EarlyStopping(patience=patience, verbose=False,filename='checkpt_dev.pt')

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, tag_pad_idx)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, tag_pad_idx)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
    
    early_stopping(valid_loss, model)
    if early_stopping.early_stop:
        print("Early stopping, reloading checkpoint model")
        model.load_state_dict(torch.load('checkpt_dev.pt'))
        break
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 2.740 | Train Acc: 38.76%
	 Val. Loss: 2.445 |  Val. Acc: 44.95%
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 2.095 | Train Acc: 46.10%
	 Val. Loss: 1.891 |  Val. Acc: 49.00%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 1.606 | Train Acc: 60.65%
	 Val. Loss: 1.524 |  Val. Acc: 59.24%
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 1.247 | Train Acc: 67.25%
	 Val. Loss: 1.231 |  Val. Acc: 66.35%
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.982 | Train Acc: 73.29%
	 Val. Loss: 1.021 |  Val. Acc: 71.28%
Epoch: 06 | Epoch Time: 0m 13s
	Train Loss: 0.812 | Train Acc: 77.15%
	 Val. Loss: 0.869 |  Val. Acc: 75.23%
Epoch: 07 | Epoch Time: 0m 13s
	Train Loss: 0.694 | Train Acc: 80.20%
	 Val. Loss: 0.763 |  Val. Acc: 77.78%
Epoch: 08 | Epoch Time: 0m 11s
	Train Loss: 0.600 | Train Acc: 82.78%
	 Val. Loss: 0.695 |  Val. Acc: 79.77%
Epoch: 09 | Epoch Time: 0m 8s
	Train Loss: 0.531 | Train Acc: 84.64%
	 Val. Loss: 0.641 |  Val. Acc: 81.57%
Epoch: 10 | Epoch Ti

In [21]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, tag_pad_idx)
print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.329 |  Test Acc: 90.78%


89.17, 90.15, 90.33, 89.21, 90.78